In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('fede'))
if module_path not in sys.path:
    sys.path.append(module_path)

from client import Client
import pandas as pd
from supported_modles import Supported_modles
import utils
from fedavg import Fedavg
from sklearn.utils.class_weight import compute_sample_weight, compute_class_weight
from sklearn.model_selection import StratifiedKFold, GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
import random

In [2]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [3]:
clients = utils.set_data(True)

Creating node1.
Creating node2.
Creating node3.
Creating node4.
Creating node5.
DOWNSAMLING node2
DOWNSAMLING node4
DOWNSAMLING node5


In [4]:
selected_model = Supported_modles.logistic_regression

In [5]:
f1_before = []
for client in clients:
    client.split_data()
    # client.downsample_data(['sbytes','dbytes','sttl','dttl','spkts','dpkts'])
    # client.prep_data()
    client.train_model(selected_model)
    f1_before.append(client.f1)

In [6]:
f1_before

[0.802195045468799,
 0.9129629629629629,
 0.8668507114036951,
 0.958904109589041,
 0.9955062120010574]

In [7]:
selected_model = Supported_modles.SGD_classifier

In [8]:
# clients = utils.set_data(True)
for client in clients:
    # client.split_data()
    client.init_empty_model(selected_model,None)

In [9]:
fedavg = Fedavg("global", 0.1)
fedavg.init_global_model(selected_model, None,78)

Waitiing for a Connection..


In [22]:
number_of_rounds = 10
batch_size = 0.2
epochs = 10

for _ in range(number_of_rounds):
    print(f'Starting new round!')

    applicable_clients = random.sample((clients), random.randint(1, 4))
    applicable_models = []
    applicable_name = []
    round_weights = []
    dataset_size = 0
    

    for client in applicable_clients:
        # print(f'Client name: {client.name}')

        X = client.x
        y = client.y

        X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=batch_size, stratify=y, random_state=random.randint(0,10))
       
        # X_train = prep.fit_transform(X_train)

        dataset_size += X_train.shape[0]
        sample_weights = compute_sample_weight('balanced', y=y_train)

        fedavg.load_global_model(client.model, selected_model)
        fedavg.train_local_agent(X_train, y_train, client.model, epochs, sample_weights, selected_model)
        round_weights.append(X_train.shape[0])
        applicable_models.append(client.model)


    round_weights = np.array(round_weights) / dataset_size # calculate weight based on actual dataset size
    print(round_weights)
    # round_weights = weights
    fedavg.update_global_model(applicable_models, round_weights, selected_model)

Starting new round!
[0.08820065 0.00658214 0.10766992 0.79754729]
Starting new round!
[0.00737635 0.09884308 0.89378057]
Starting new round!
[0.48146531 0.06167646 0.45685823]
Starting new round!
[0.47965679 0.06144478 0.45514215 0.00375628]
Starting new round!
[1.]
Starting new round!
[0.08820065 0.79754729 0.10766992 0.00658214]
Starting new round!
[0.11355499 0.88644501]
Starting new round!
[0.88033382 0.11277213 0.00689405]
Starting new round!
[0.89378057 0.00737635 0.09884308]
Starting new round!
[1.]


In [23]:
f1_fedavg = []
for client in clients:
    # x = prep.transform(client.x_test)
    y_hat = fedavg.model.predict(client.x_test)
    f1_fedavg.append(f1_score(client.y_test,y_hat))
print(f1_fedavg)

[0.8683030015682556, 0.821202966218072, 0.7894388468044421, 0.7813953488372092, 0.6216022889842632]


In [24]:
i = 0
for client in clients:
    print(f'Difference {f1_fedavg[i]-f1_before[i]}')
    i += 1

Difference 0.06610795609945652
Difference -0.09175999674489088
Difference -0.07741186459925298
Difference -0.17750876075183186
Difference -0.3739039230167942
